In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.utils.training_utils import multi_gpu_model

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import Xception
from keras.applications.xception import preprocess_input

Using TensorFlow backend.


In [2]:
#TOT = 35534421
TOT = 46613580
EPOCHS = 50
size = 71
batchsize = 680
lw = 6
channel = 3
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = False
optimizer = 'adam'
cb = 'raw'

In [3]:
K.clear_session()
# model = Xception(input_shape=(size, size, channel), weights=None, classes=NCATS)
model = Xception(input_shape=(None,None,3), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model = multi_gpu_model(model, gpus=2)

# model.load_weights('models/xception71_parts2_lw6_balance_0_adam_color.model')

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
xception (Model)                (None, 340)          21558140    lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
__________

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd_parts_threshold(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center, detail_threshold = 5)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

x shape (34000, 71, 71, 3) x max 255


In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 71, 71, 3) (34000, 340)
Validation array memory 1.92 GB


In [7]:
train_datagen = image_generator_xd_parts_threshold(df_path = '../input/train_all.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                   channel=channel, mixup=mixup, center=center,
                                   detail_threshold = 5)

In [8]:
model_prefix = 'xception{}_threshparts_lw{}_balance_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
if channel == 3:
    model_prefix += '_color'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', 
                                  save_best_only=True, save_weights_only=True,
                                  verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=3,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

xception71_threshparts_lw6_balance_0_adam_color


In [9]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/30
2285/2284 [==============================] - 1383s 605ms/step - loss: 1.6619 - categorical_crossentropy: 1.6619 - categorical_accuracy: 0.6014 - top_3_accuracy: 0.7818 - val_loss: 1.3466 - val_categorical_crossentropy: 1.3466 - val_categorical_accuracy: 0.6629 - val_top_3_accuracy: 0.8426

Epoch 00001: val_top_3_accuracy improved from -inf to 0.84256, saving model to ./models/xception71_threshparts_lw6_balance_0_adam_color.model
Epoch 2/30
2285/2284 [==============================] - 1399s 612ms/step - loss: 1.1411 - categorical_crossentropy: 1.1411 - categorical_accuracy: 0.7103 - top_3_accuracy: 0.8725 - val_loss: 1.1751 - val_categorical_crossentropy: 1.1751 - val_categorical_accuracy: 0.7031 - val_top_3_accuracy: 0.8670

Epoch 00002: val_top_3_accuracy improved from 0.84256 to 0.86697, saving model to ./models/xception71_threshparts_lw6_balance_0_adam_color.model
Epoch 3/30
2285/2284 [==============================] - 1394s 610ms/step - loss: 1.0421 - categorical_crossen

2285/2284 [==============================] - 1370s 599ms/step - loss: 0.7649 - categorical_crossentropy: 0.7649 - categorical_accuracy: 0.7993 - top_3_accuracy: 0.9253 - val_loss: 0.8005 - val_categorical_crossentropy: 0.8005 - val_categorical_accuracy: 0.7911 - val_top_3_accuracy: 0.9217

Epoch 00020: val_top_3_accuracy did not improve from 0.92171
Epoch 21/30
2285/2284 [==============================] - 1369s 599ms/step - loss: 0.7641 - categorical_crossentropy: 0.7641 - categorical_accuracy: 0.7995 - top_3_accuracy: 0.9253 - val_loss: 0.8307 - val_categorical_crossentropy: 0.8307 - val_categorical_accuracy: 0.7832 - val_top_3_accuracy: 0.9171

Epoch 00021: val_top_3_accuracy did not improve from 0.92171
Epoch 22/30
2285/2284 [==============================] - 1369s 599ms/step - loss: 0.7610 - categorical_crossentropy: 0.7610 - categorical_accuracy: 0.8004 - top_3_accuracy: 0.9261 - val_loss: 0.7864 - val_categorical_crossentropy: 0.7864 - val_categorical_accuracy: 0.7958 - val_top_3

KeyboardInterrupt: 

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


In [ ]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [ ]:
doodle_predict(model, check_path, x_test)